# Description

### Read annotated nodes data

In [ ]:
import pandas as pd
import os

ANNOTATED_NODES_CSV = "reachable-nodes-annotated-2023-05-07.csv"

assert os.path.exists(ANNOTATED_NODES_CSV), "Annotated node data missing! Run 01_prep_input_data.ipynb to create."

df = pd.read_csv(ANNOTATED_NODES_CSV)

# address statistics
num_ipv4 = sum(df.network == 'ipv4')
num_ipv6 = sum(df.network == 'ipv6')
num_ip = len(df)
assert num_ipv4 + num_ipv6 == num_ip, "Inconsistent dichotomy"
print(f'reachable nodes: ipv4={num_ipv4}, ipv6={num_ipv6}, ipv4/ipv6={num_ip}')

# netgroup statistics
ip_netgroups = df.netgroup.value_counts().values
ipv4_netgroups = df[df.network == 'ipv4'].netgroup.value_counts().values
ipv6_netgroups = df[df.network == 'ipv6'].netgroup.value_counts().values
assert sum(ipv4_netgroups) + sum(ipv6_netgroups) + sum(ip_netgroups) == 2 * len(df), "Inconsistent dichotomy!"
print(f'netgroups: ipv4={len(ipv4_netgroups)}, ipv6={len(ipv6_netgroups)}, ipv4/ipv6={len(ip_netgroups)}, total_addresses={sum(ip_netgroups)}')

# asn statistics
ip_asns = df.asn.value_counts().values
ipv4_asns = df[df.network == 'ipv4'].asn.value_counts().values
ipv6_asns = df[df.network == 'ipv6'].asn.value_counts().values
assert sum(ipv4_asns) + sum(ipv6_asns) + sum(ip_asns) == 2 * len(df), "Inconsistent dichotomy!"
print(f'asn: ipv4={len(ipv4_asns)}, ipv6={len(ipv6_asns)}, ipv4/ipv6={len(ip_asns)}, total_addresses={sum(ip_asns)}')

## Validation of analytic model with Monte Carlo

- Speed-accuracy tradeoff can be tuned with `MONTECARLO_ITERATIONS`
- Comparison carried out with data from netgroup policy

In [ ]:
import numpy as np
import seaborn as sns
from itertools import product
from tqdm import tqdm
tqdm.pandas()

# BiasedUrn R package
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import IntVector, FloatVector
import rpy2.robjects as robjects
biasedurn = rpackages.importr('BiasedUrn')

def p_eclipse_montecarlo(good_bucket_sizes: list[int], bad_bucket_size: float, num_bad_buckets: int, num_outbound_conns: int = 10) -> float:
    """
    Compute eclipse probability (i.e. all outbound to attacker nodes) using a
    Monte Carlo simulation. Cache results to avoid redundant lengthy computations.
    """
    bad_bucket_sizes = tuple([bad_bucket_size]) * num_bad_buckets
    bucket_sizes = np.array(good_bucket_sizes + bad_bucket_sizes)
    a = np.arange(len(bucket_sizes))        # population from which samples are drawn
    size = num_outbound_conns               # number of samples
    replace = False                         # sample without replacements
    p = bucket_sizes/bucket_sizes.sum()     # probabilities associated with entries in a

    count = 0
    for _ in range(MONTECARLO_ITERATIONS):
        draw = np.random.choice(a, size, replace, p=bucket_sizes/bucket_sizes.sum())
        if np.sum(draw >= len(good_bucket_sizes)) == num_outbound_conns:
            count += 1

    p = count / MONTECARLO_ITERATIONS
    return p

def p_eclipse_wallenius(good_bucket_sizes: tuple[int], bad_bucket_size: float, num_bad_buckets: int, num_outbound_conns: int = 10) -> float:
    x = IntVector([0] * len(good_bucket_sizes) + [num_outbound_conns])  # number of balls of each color sampled: zero benign balls, number of outgoing conns. bad balls
    m = IntVector([1] * len(good_bucket_sizes) + [num_bad_buckets])     # initial number of balls of each color: one ball for each benign color, number of buckets balls for bad color
    n = num_outbound_conns                                              # number of balls sampled: number of outbound connections
    odds = FloatVector(good_bucket_sizes + tuple([bad_bucket_size]))    # weight for each color, arbitrarily scaled: number of nodes in bucket
    p = biasedurn.dMWNCHypergeo(x=x, m=m, n=n, odds=odds)               # returns an R FloatVector that contains one element
    return p[0]


# define parameters
MONTECARLO_ITERATIONS = 50000 # 50000 was used for the chart in the article
good_netgroup_sizes = [ipv4_netgroups]
num_bad_nodes = [20000, 30000]
netgroup_ratio = np.logspace(np.log10(0.001), np.log10(0.3), num=40, base=10)
df_netgroup = pd.DataFrame(list(product(good_netgroup_sizes, num_bad_nodes, netgroup_ratio)), columns=['good_netgroup_sizes', 'num_bad_nodes', 'netgroup_ratio'])
df_netgroup['num_bad_netgroups'] = (df_netgroup.num_bad_nodes * df_netgroup.netgroup_ratio).astype(int)
df_netgroup['bad_netgroup_size'] = df_netgroup.num_bad_nodes / df_netgroup.num_bad_netgroups

# run parameter studies
df_netgroup['p_montecarlo'] = df_netgroup.progress_apply(lambda x: p_eclipse_montecarlo(tuple(x.good_netgroup_sizes), x.bad_netgroup_size, x.num_bad_netgroups), axis=1)
df_netgroup['p_wallenius'] = df_netgroup.apply(lambda x: p_eclipse_wallenius(tuple(x.good_netgroup_sizes), x.bad_netgroup_size, x.num_bad_netgroups), axis=1)

# plot (analytic results use lines; mc validation uses x marker)
_ = sns.lineplot(data=df_netgroup, x='num_bad_netgroups', y='p_wallenius', hue='num_bad_nodes', legend=False)
g = sns.scatterplot(data=df_netgroup, x='num_bad_netgroups', y='p_montecarlo', hue='num_bad_nodes', marker='x')
g.set_xscale('log')

## Demonstrate distribution-independence of eclipse probabilities

- Demonstrate `p_eclipse` is identical for netgroup and ASN bucketing (i.e., independent of the distribution of benign nodes across buckets)

In [ ]:
# define parameters
num_bad_nodes = range(10000, 50000, 5000)
num_bad_buckets = range(10, 200, 10)

df_in = pd.DataFrame(list(product(num_bad_nodes, num_bad_buckets)), columns=['num_bad_nodes', 'num_bad_buckets'])
df_in['bad_bucket_size'] = df_in.num_bad_nodes / df_in.num_bad_buckets
df_in['good_netgroup_sizes'] = [tuple(ip_netgroups)] * len(df_in)
df_in['good_asn_sizes'] = [tuple(ip_asns)] * len(df_in)

# run parameter studies
df_in['p_netgroup'] = df_in.progress_apply(lambda x: p_eclipse_wallenius(tuple(x.good_netgroup_sizes), x.bad_bucket_size, x.num_bad_buckets), axis=1)
df_in['p_asn'] = df_in.progress_apply(lambda x: p_eclipse_wallenius(tuple(x.good_asn_sizes), x.bad_bucket_size, x.num_bad_buckets), axis=1)


# plot (analytic results use lines; mc validation uses x marker)
_ = sns.lineplot(data=df_in, x='num_bad_buckets', y='p_netgroup', hue='num_bad_nodes', legend=False)
g = sns.scatterplot(data=df_in, x='num_bad_buckets', y='p_asn', hue='num_bad_nodes', marker='x')
g.set_ylabel('Number of attacker buckets')

### Demonstrate validity of transformations of distribution of benign nodes

- Map all benign nodes into a single bucket before calling function that computes Wallenius function and compare to vanilla results

In [ ]:
def p_eclipse_wallenius_fast(good_bucket_sizes: tuple[int], bad_bucket_size: float, num_bad_buckets: int, num_outbound_conns: int = 10) -> float:
    x = IntVector([0] + [num_outbound_conns])  # number of balls of each color sampled: zero benign balls, number of outgoing conns. bad balls
    m = IntVector([1] + [num_bad_buckets])     # initial number of balls of each color: one ball for each benign color, number of buckets balls for bad color
    n = num_outbound_conns                     # number of balls sampled: number of outbound connections
    odds = FloatVector(tuple([sum(good_bucket_sizes)]) + tuple([bad_bucket_size]))  # weight for each color, arbitrarily scaled: number of nodes in bucket
    p = biasedurn.dMWNCHypergeo(x=x, m=m, n=n, odds=odds)   # returns an R FloatVector that contains one element
    return p[0]

good_netgroup_sizes = [ipv4_netgroups]
num_bad_nodes = [20000, 30000]
netgroup_ratio = np.logspace(np.log10(0.001), np.log10(0.3), num=40, base=10)
df_in = pd.DataFrame(list(product(good_netgroup_sizes, num_bad_nodes, netgroup_ratio)), columns=['good_netgroup_sizes', 'num_bad_nodes', 'netgroup_ratio'])
df_in['num_bad_netgroups'] = (df_in.num_bad_nodes * df_in.netgroup_ratio).astype(int)
df_in['bad_netgroup_size'] = df_in.num_bad_nodes / df_in.num_bad_netgroups

# run parameter studies
df_in['p_wallenius_fast'] = df_in.progress_apply(lambda x: p_eclipse_wallenius_fast(tuple(x.good_netgroup_sizes), x.bad_netgroup_size, x.num_bad_netgroups), axis=1)
df_in['p_wallenius'] = df_in.progress_apply(lambda x: p_eclipse_wallenius(tuple(x.good_netgroup_sizes), x.bad_netgroup_size, x.num_bad_netgroups), axis=1)

# plot (analytic results use lines; mc validation uses x marker)
_ = sns.lineplot(data=df_in, x='num_bad_netgroups', y='p_wallenius', hue='num_bad_nodes', legend=False)
g = sns.scatterplot(data=df_in, x='num_bad_netgroups', y='p_wallenius_fast', hue='num_bad_nodes', marker='x')
g.set_xscale('log')